<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/Zero_shot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 59.9 MB/s 
     |████████████████████████████████| 1.2 MB 58.0 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 


In [2]:
from transformers import pipeline
import pandas as pd

In [3]:
classifier = pipeline("zero-shot-classification", # transfer learning
                      model = "joeddav/xlm-roberta-large-xnli", # multi-lingual model
                      device = 0) # to utilize GPU

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [10]:
import re
from datetime import datetime
import os.path
from os import path
import requests


ticker_path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/Tickers_reduced_full.csv"


tickers = pd.read_csv(ticker_path)
tickers_list = tickers["Tickers"].tolist()

path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/"

In [5]:
def analyse(ticker):
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  df = pd.read_csv(file_path)

  texts = df["Text_clean"]

  chunks = [texts[x : x + 100] for x in range(0, len(texts), 100)] # split into chunks

  results_list = []

  for idx in range(len(chunks)):

    cp1 = datetime.now()
    
    chunk = chunks[idx].tolist()

    candidate_labels = ["positive", "negative", "neutral"]

    hypothesis_template = "The sentiment of this tweet is {}."

    #dicts = classifier(text, candidate_labels, hypothesis_template = hypothesis_template)
    #result = dicts["labels"][0]
    #results_list.append(result)
    
    list_results_dicts = classifier(chunk, candidate_labels, hypothesis_template = hypothesis_template)
    new_list = [i["labels"][0] for i in list_results_dicts]
    results_list.extend(new_list)

    cp2 = datetime.now()

    print(str(idx + 1) + str("/") + str(len(chunks)) + str(" --- ") + str(cp2 - cp1))

  return results_list

In [12]:
tickers_list = tickers_list[2:]

In [ ]:
for ticker in tickers_list:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
 
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)
  
  else:

    print(ticker + str(" is missing"))

    continue

ACA is present
1/20 --- 0:00:02.765079
2/20 --- 0:00:02.749449
3/20 --- 0:00:03.036106
4/20 --- 0:00:02.618832
5/20 --- 0:00:02.930263
6/20 --- 0:00:02.872553
7/20 --- 0:00:02.591838
8/20 --- 0:00:02.244237
9/20 --- 0:00:02.799577
10/20 --- 0:00:02.856396
11/20 --- 0:00:02.461212
12/20 --- 0:00:02.446333
13/20 --- 0:00:02.362803
14/20 --- 0:00:02.606407
15/20 --- 0:00:02.855243
16/20 --- 0:00:03.017965
17/20 --- 0:00:02.272795
18/20 --- 0:00:02.765288
19/20 --- 0:00:02.851491
20/20 --- 0:00:00.890980
ADS is present
1/74 --- 0:00:02.606028
2/74 --- 0:00:02.761881
3/74 --- 0:00:02.904708
4/74 --- 0:00:02.875723
5/74 --- 0:00:03.061722
6/74 --- 0:00:03.111707
7/74 --- 0:00:03.118822
8/74 --- 0:00:03.181907
9/74 --- 0:00:03.097836
10/74 --- 0:00:03.183221
11/74 --- 0:00:03.184253
12/74 --- 0:00:03.182410
13/74 --- 0:00:03.198231
14/74 --- 0:00:03.176119
15/74 --- 0:00:03.175259
16/74 --- 0:00:03.177626
17/74 --- 0:00:03.176352
18/74 --- 0:00:02.701887
19/74 --- 0:00:02.587769
20/74 --- 0:0